# Analysis: do a song's lyrics inform playlist formation?

*Do distances between lyrics embeddings correlate with distances between song embeddings?*

**Lucas De Oliveira**

**ML Algorithms:** Word2Vec, LinearRegression

## Computing word embeddings

We have previously saved lyrics for about 138,000 unique songs in the Spotify playlist data. We will perform the following tasks:

* Load data frame from mongoDB
* Extract the words from the lyrics
* Use Word2Vec to train word embeddings
* For each song, take an average of the embeddings for each word in its lyrics as the "lyrical embedding" of the song
* Save this in a Spark DataFrame to MongoDB

In [ ]:
# Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

# Start Spark session
spark = SparkSession.builder\
                    .appName("spotify")\
                    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
                    .config("spark.mongodb.input.uri", "mongodb+srv://**********")\
                    .config("spark.mongodb.output.uri", "mongodb+srv://**********")\
                    .config("spark.network.timeout", "7200s")\
                    .config("spark.executor.heartbeatInterval", "1200s")\
                    .config("spark.databricks.adaptive.autoOptimizeShuffle.enabled", True)\
                    .getOrCreate()

# MongoDB connection info
database = '*****'
collection = 'songs_lyrics'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [ ]:
# Read song lyrics data from MongoDB (pre-processed)
df = spark.read.format('mongo').option("uri", connection_string).load()
df.show(5)

+--------------------+----------------+--------------------+--------------+--------------------+-----------+------+------------+-------------+-----------+-------------+----------+------+---+---------------+--------------------+
 _id| album_name| album_uri| artist_name| artist_uri|duration_ms|lyrics|lyrics_words| name|num_artists|num_followers|num_tracks| pid|pos| track_name| track_uri|
+--------------------+----------------+--------------------+--------------+--------------------+-----------+------+------------+-------------+-----------+-------------+----------+------+---+---------------+--------------------+
{621bb23d174bc05a...| A Woman Like Me|spotify:album:1gF...|Bettye Lavette|spotify:artist:55...| 14637735| null| null| BLU| 29| 1| 65|544620| 61|Thru The Winter|spotify:track:007...|
{621bb23d174bc05a...| St. X|spotify:album:4t9...| St. X|spotify:artist:69...| 34378081| null| null|something new| 44| 1| 156|824425| 47| Machu Picchu|spotify:track:008...|
{621bb23d174bc05a...|Heart of a Gypsy|spotify:album:6WY...| Govi|spotify:artist:72...| 58573861| null| null| relax| 56| 3| 189|729624|156| Pied Piper|spotify:track:00G...|
{621bb23d174bc05a...| Equipo Armado|spotify:album:2lL...| El Fantasma|spotify:artist:0m...| 11486651| null| null| corridos | 21| 1| 62|808464| 57| Fiel Companero|spotify:track:00I...|
{621bb23d174bc05a...| Equipo Armado|spotify:album:2lL...| El Fantasma|spotify:artist:0m...| 27437400| null| null| 1738.| 52| 1| 152| 65314| 69| Fiel Companero|spotify:track:00I...|
+--------------------+----------------+--------------------+--------------+--------------------+-----------+------+------------+-------------+-----------+-------------+----------+------+---+---------------+--------------------+
only showing top 5 rows

In the following code blocks we first inspect the lyrics DataFrame, then we train word embeddings using Word2Vec on a subset of the data in order to get the pipeline for the full data.

In [ ]:
df_lyrics = df.select("track_uri", "lyrics_words").distinct().where(df['lyrics_words'].isNotNull()).cache()

In [ ]:
print(df_lyrics.count())
df_lyrics.show(5)

138123
+--------------------+--------------------+
 track_uri| lyrics_words|
+--------------------+--------------------+
spotify:track:00d...|[pulled, the, fuc...|
spotify:track:00Y...|[but, it, would, ...|
spotify:track:5Gx...|[niggas, wanna, w...|
spotify:track:4cH...|[sure, it's, a, c...|
spotify:track:5cG...|[jinsei, jinsei, ...|
+--------------------+--------------------+
only showing top 5 rows

In [ ]:
df_test = df_lyrics.limit(10).cache()
df_test.show()

+--------------------+--------------------+
 track_uri| lyrics_words|
+--------------------+--------------------+
spotify:track:00w...|[when, the, light...|
spotify:track:004...|[caressing, our, ...|
spotify:track:00K...|[separate,, sifti...|
spotify:track:0CK...|[one, more, day, ...|
spotify:track:0EM...|[sounded, sad, up...|
spotify:track:0Da...|[[secretary:], , ...|
spotify:track:0G5...|[yo, con, mis, br...|
spotify:track:0Ih...|[but, our, local,...|
spotify:track:09U...|[wish, i, couldn'...|
spotify:track:060...|[don't, you, dare...|
+--------------------+--------------------+

In [ ]:
word2Vec = Word2Vec(vectorSize=32, seed=42, inputCol="lyrics_words", outputCol="model")
word2Vec.setMaxIter(10)
model = word2Vec.fit(df_test)

In [ ]:
vectors_test = model.getVectors().cache()
vectors_test.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
+-----------+--------------------+
 word| vector|
+-----------+--------------------+
 down|[-0.0505827143788...|
 it's|[0.30985769629478...|
 |[0.02132896520197...|
 for|[0.41402989625930...|
 people|[-0.1829547435045...|
 this|[0.39972665905952...|
 (well|[0.43436864018440...|
 in|[0.20833760499954...|
 have|[-0.0325801782310...|
 your|[0.32904917001724...|
 off|[0.64350163936614...|
 are|[-0.2755229771137...|
 is|[0.11786556243896...|
 oh|[0.19284033775329...|
 stars|[-0.0939949974417...|
 try|[0.18134884536266...|
 dress|[0.29417058825492...|
 tu|[-0.1316887140274...|
loo-rye-aye|[0.25482434034347...|
 you're|[0.18157753348350...|
+-----------+--------------------+
only showing top 20 rows

In [ ]:
# Fit Word2Vec on entire dataframe (original tokenization with Tokenizer in preprocessing)
word2Vec = Word2Vec(vectorSize=32, seed=42, inputCol="lyrics_words", outputCol="model")
word2Vec.setMaxIter(10)
model = word2Vec.fit(df_lyrics)

In [ ]:
word_vectors = model.getVectors().cache()

In [ ]:
word_vectors.show(10)

+----------+--------------------+
 word| vector|
+----------+--------------------+
[teefliii]|[0.06319652497768...|
 knees...|[0.11822315305471...|
 next?)|[-0.1456330865621...|
 spinnin'|[0.28110203146934...|
 exodus!|[-0.4918343126773...|
 ohohoho|[-0.1712255775928...|
 come?|[0.12748654186725...|
 echo.|[0.27712708711624...|
 worth...|[0.18746957182884...|
 stegen|[0.24622718989849...|
+----------+--------------------+
only showing top 10 rows

The words created by the Tokenizer during the preprocessing stage ends up giving us some strange results (see DataFrame above). Instead, we'll grab the lyrics from the DataFrame and directly extract our own words using the **lyrics2words** function below.

In [ ]:
# Function to clean words
def lyrics2words(lyrics):
    chars = [c for c in lyrics.lower() if c.isalnum() or c.isspace()]
    return ''.join(chars).split()

To make sure that our function can handle non-English characters, we inspect the following:

In [ ]:
'ã'.isalnum()

Out[12]: True

In [ ]:
'諺文'.isalnum()

Out[13]: True

Next, we subset only songs with lyrics and apply the lyrics2songs function to extract a words column.

In [ ]:
# Distinct rows for songs with lyrics
songs_w_lyrics = df.select("track_uri", "lyrics").distinct().where(df['lyrics_words'].isNotNull()).cache()
songs_w_lyrics.show(5)

+--------------------+--------------------+
 track_uri| lyrics|
+--------------------+--------------------+
spotify:track:003...|I wear it like a ...|
spotify:track:00J...|Meglio che ti lib...|
spotify:track:00J...|Only time until,\...|
spotify:track:00W...|I'd like to thank...|
spotify:track:00X...|So why the ghetto...|
+--------------------+--------------------+
only showing top 5 rows

In [ ]:
udf_lyrics2songs = udf(lambda x: lyrics2words(x), returnType=ArrayType(StringType()))
songs_df = songs_w_lyrics.select("track_uri", "lyrics", udf_lyrics2songs(songs_w_lyrics['lyrics']).alias("words")).cache()

In [ ]:
songs_df.show(5)

+--------------------+--------------------+--------------------+
 track_uri| lyrics| words|
+--------------------+--------------------+--------------------+
spotify:track:003...|I wear it like a ...|[i, wear, it, lik...|
spotify:track:00J...|Meglio che ti lib...|[meglio, che, ti,...|
spotify:track:00J...|Only time until,\...|[only, time, unti...|
spotify:track:00W...|I'd like to thank...|[id, like, to, th...|
spotify:track:00X...|So why the ghetto...|[so, why, the, gh...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

In [ ]:
# Fit Word2Vec on new 
word2Vec = Word2Vec(vectorSize=32, seed=42, inputCol="words", outputCol="model")
word2Vec.setMaxIter(10)
model = word2Vec.fit(songs_df)

In [ ]:
word_vectors = model.getVectors().cache()

In [ ]:
word_vectors.show(5)

+---------+--------------------+
 word| vector|
+---------+--------------------+
 ohohoho|[-0.0792507305741...|
professed|[-0.2632994353771...|
 stegen|[0.56281369924545...|
 acéptalo|[0.73900830745697...|
 jaleo|[0.42074951529502...|
+---------+--------------------+
only showing top 5 rows

In [ ]:
word_vectors.count()

Out[20]: 87733

In [ ]:
word_vectors.printSchema()

root
-- word: string (nullable = true)
-- vector: vector (nullable = true)

In [ ]:
# Create UDF to convert the vector to an array for storage in MongoDB
vec2arr = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

In [ ]:
wordvec_df = word_vectors.select("word", vec2arr(word_vectors["vector"]).alias("vector"))

In [ ]:
wordvec_df.show(5)

+---------+--------------------+
 word| vector|
+---------+--------------------+
 ohohoho|[-0.07925073, 0.3...|
professed|[-0.26329944, 0.1...|
 stegen|[0.5628137, 1.032...|
 acéptalo|[0.7390083, 0.214...|
 jaleo|[0.42074952, -0.1...|
+---------+--------------------+
only showing top 5 rows

In [ ]:
wordvec_df.printSchema()

root
-- word: string (nullable = true)
-- vector: array (nullable = true)
 |-- element: float (containsNull = true)

Because we have already saved the word vectors to mongoDB, we commented out the code below. Uncomment to do this the first time.

In [ ]:
# Uncomment to save word, vector pairs in mongodb
# collection = 'word_vectors'
# connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
# wordvec_df.write.format("mongo").option("uri", connection_string).mode("append").save()

### Computing lyrics embeddings

Below we compute the average lyrical embedding for each song as the average of the embeddings for all words in the lyrics.

In [ ]:
# Get word vector dictionary for faster processing
word_vec_dict = wordvec_df.toPandas().set_index("word").to_dict()
word_vecs = word_vec_dict['vector']
word_vecs

Out[27]: {'ohohoho': array([-0.07925073, 0.31392673, -0.23940067, 0.09624004, 0.10937865,
 -0.03422616, -1.1821041 , -0.50068593, 0.28972298, 0.06530909,
 -1.23165 , 0.04650208, -0.9326481 , 0.64062274, -1.14963 ,
 -0.36210456, 0.16882792, -0.422122 , -0.329844 , -0.05391987,
 0.02306327, 0.27957904, 0.7619971 , 0.5245532 , -0.28863308,
 0.1532805 , 0.10613267, -0.17104477, -0.5588746 , -0.3626776 ,
 -0.31239626, -0.34137258], dtype=float32),
 'professed': array([-0.26329944, 0.14257132, 0.01223039, -0.25976172, 0.34392947,
 0.08153762, -0.3767318 , -0.05467109, 0.22877507, -0.06349373,
 0.3972084 , 0.35623866, -0.07845655, 0.22269817, -0.38570687,
 -0.25012845, -0.26922402, -0.33298013, 0.1100001 , -0.17502022,
 -0.54777193, -0.18024419, -0.22611459, -0.14369097, -0.4080993 ,
 0.06734408, 0.12605414, -0.06174215, -0.17894982, -0.4430647 ,
 -0.27485955, 0.17589772], dtype=float32),
 'stegen': array([ 0.5628137 , 1.0322635 , 0.82646424, 1.4237982 , -0.10312799,
 1.752393 , -1.1530322 , -1.1665572 , -0.6774264 , 0.16445562,
 -0.27869236, -0.28455666, -0.23084715, -0.4388019 , -1.0828527 ,
 0.17807034, 0.28453764, -1.0028799 , 0.42642018, -0.42699164,
 0.6356657 , -0.35567403, 0.5775872 , 0.43228108, -0.23928046,
 0.42356327, 0.3333311 , -0.59891206, 0.11412388, 0.14864314,
 0.6790861 , -0.8523481 ], dtype=float32),
 'acéptalo': array([ 0.7390083 , 0.2149252 , 0.98295504, -0.55815846, -0.27640525,
 0.3235611 , -1.1656953 , -0.95920527, 0.24490897, -0.14984235,
 -0.56612056, 0.2153208 , -0.67097735, -0.77171826, 0.25893044,
 -1.063229 , 0.13935147, -0.13208862, 0.10008043, 0.10712686,
 1.042984 , 0.08250144, -0.2776759 , 0.08776684, 0.57545274,
 -0.04026667, -0.21765359, -0.24504882, -0.10052905, -0.0739845 ,
 0.26224092, 0.29016572], dtype=float32),
 'jaleo': array([ 0.42074952, -0.1761267 , 0.6410343 , -0.09129529, -0.36618516,
 0.6331861 , -0.33989832, -0.34172627, -0.05461185, -0.18391614,
 0.11863644, 0.40089455, -0.5235228 , -0.53738797, -0.03182067,
 -1.6687939 , -0.07918644, -0.07320985, 0.13400044, 0.597256 ,
 0.84270275, 0.02402407, -0.22340074, 0.2525892 , 0.75876826,
 0.24889816, 0.18219714, 0.1889677 , 0.15539242, 0.26814985,
 0.15813926, 0.42751363], dtype=float32),
 'admito': array([ 0.59465104, -0.03291072, 0.8942622 , -0.07866066, -0.48475948,
 1.1740804 , -0.10754308, -0.912207 , -0.13435338, -0.3679157 ,
 -0.01501329, 0.35489586, -0.09476306, -0.74322563, 0.51321924,
 -1.0066966 , 0.44180095, 0.3592743 , -0.29224554, 0.3905961 ,
 0.06380587, -0.10512977, 0.2179836 , 0.29496008, 0.77602106,
 -0.51328236, -0.3167829 , -0.362109 , 0.22239216, 0.20878783,
 -0.02232932, 0.41146603], dtype=float32),
 'dredd': array([ 0.02094032, -0.14081368, 0.4646859 , -0.1802023 , -0.09167867,
 -0.0127033 , -0.07945038, -0.37925902, -0.11891985, 0.1520347 ,
 0.39303723, 0.17207187, 0.32025346, 0.18145931, -0.35374242,
 -0.19798227, 0.21809755, -0.2292551 , 0.3739225 , -0.03289745,
 0.03386633, -0.8707283 , -0.0642144 , -0.5261853 , -0.5366441 ,
 -0.32214195, -0.27079347, 0.36429778, -0.04926702, -0.06325061,
 -0.4224059 , -0.04141013], dtype=float32),
 'gaiden': array([ 0.03694442, -0.00840622, 0.40425175, 0.1387582 , -0.23771454,
 0.16233376, 0.03707598, -0.5078011 , 0.23291597, 0.04278218,
 0.22757229, -0.04662805, 0.73560065, 0.0654904 , -0.3404447 ,
 -0.27189517, -0.02874799, -0.04407474, -0.13808799, -0.07573457,
 0.3978408 , -0.64466333, -0.06974618, -0.27431408, -0.08188342,
 0.07534437, -0.2436077 , 0.21145846, 0.125808 , 0.25228682,
 0.38125023, 0.43964297], dtype=float32),
 'protegido': array([ 0.56665003, 0.02630041, 0.79432154, -0.1659684 , -0.2770339 ,
 0.5420451 , -0.13389336, -0.5671256 , -0.03399916, -0.3362539 ,
 -0.20322956, 0.5491727 , -0.07410609, -0.6306755 , 0.2925735 ,
 -1.422273 , -0.14492823, -0.31507894, -0.26647982, 0.50701994,
 0.6172955 , -0.341631 , -0.11572693, 0.22890377, 0.55911905,
 -0.02285498, -0.24403843, -0.04549028, 0.20241591, 0.25729093,
 0.06621232, 0.2978325 ], dtype=float32),
 'advertencia

In [ ]:
word_vecs

Out[28]: {'ohohoho': array([-0.07925073, 0.31392673, -0.23940067, 0.09624004, 0.10937865,
 -0.03422616, -1.1821041 , -0.50068593, 0.28972298, 0.06530909,
 -1.23165 , 0.04650208, -0.9326481 , 0.64062274, -1.14963 ,
 -0.36210456, 0.16882792, -0.422122 , -0.329844 , -0.05391987,
 0.02306327, 0.27957904, 0.7619971 , 0.5245532 , -0.28863308,
 0.1532805 , 0.10613267, -0.17104477, -0.5588746 , -0.3626776 ,
 -0.31239626, -0.34137258], dtype=float32),
 'professed': array([-0.26329944, 0.14257132, 0.01223039, -0.25976172, 0.34392947,
 0.08153762, -0.3767318 , -0.05467109, 0.22877507, -0.06349373,
 0.3972084 , 0.35623866, -0.07845655, 0.22269817, -0.38570687,
 -0.25012845, -0.26922402, -0.33298013, 0.1100001 , -0.17502022,
 -0.54777193, -0.18024419, -0.22611459, -0.14369097, -0.4080993 ,
 0.06734408, 0.12605414, -0.06174215, -0.17894982, -0.4430647 ,
 -0.27485955, 0.17589772], dtype=float32),
 'stegen': array([ 0.5628137 , 1.0322635 , 0.82646424, 1.4237982 , -0.10312799,
 1.752393 , -1.1530322 , -1.1665572 , -0.6774264 , 0.16445562,
 -0.27869236, -0.28455666, -0.23084715, -0.4388019 , -1.0828527 ,
 0.17807034, 0.28453764, -1.0028799 , 0.42642018, -0.42699164,
 0.6356657 , -0.35567403, 0.5775872 , 0.43228108, -0.23928046,
 0.42356327, 0.3333311 , -0.59891206, 0.11412388, 0.14864314,
 0.6790861 , -0.8523481 ], dtype=float32),
 'acéptalo': array([ 0.7390083 , 0.2149252 , 0.98295504, -0.55815846, -0.27640525,
 0.3235611 , -1.1656953 , -0.95920527, 0.24490897, -0.14984235,
 -0.56612056, 0.2153208 , -0.67097735, -0.77171826, 0.25893044,
 -1.063229 , 0.13935147, -0.13208862, 0.10008043, 0.10712686,
 1.042984 , 0.08250144, -0.2776759 , 0.08776684, 0.57545274,
 -0.04026667, -0.21765359, -0.24504882, -0.10052905, -0.0739845 ,
 0.26224092, 0.29016572], dtype=float32),
 'jaleo': array([ 0.42074952, -0.1761267 , 0.6410343 , -0.09129529, -0.36618516,
 0.6331861 , -0.33989832, -0.34172627, -0.05461185, -0.18391614,
 0.11863644, 0.40089455, -0.5235228 , -0.53738797, -0.03182067,
 -1.6687939 , -0.07918644, -0.07320985, 0.13400044, 0.597256 ,
 0.84270275, 0.02402407, -0.22340074, 0.2525892 , 0.75876826,
 0.24889816, 0.18219714, 0.1889677 , 0.15539242, 0.26814985,
 0.15813926, 0.42751363], dtype=float32),
 'admito': array([ 0.59465104, -0.03291072, 0.8942622 , -0.07866066, -0.48475948,
 1.1740804 , -0.10754308, -0.912207 , -0.13435338, -0.3679157 ,
 -0.01501329, 0.35489586, -0.09476306, -0.74322563, 0.51321924,
 -1.0066966 , 0.44180095, 0.3592743 , -0.29224554, 0.3905961 ,
 0.06380587, -0.10512977, 0.2179836 , 0.29496008, 0.77602106,
 -0.51328236, -0.3167829 , -0.362109 , 0.22239216, 0.20878783,
 -0.02232932, 0.41146603], dtype=float32),
 'dredd': array([ 0.02094032, -0.14081368, 0.4646859 , -0.1802023 , -0.09167867,
 -0.0127033 , -0.07945038, -0.37925902, -0.11891985, 0.1520347 ,
 0.39303723, 0.17207187, 0.32025346, 0.18145931, -0.35374242,
 -0.19798227, 0.21809755, -0.2292551 , 0.3739225 , -0.03289745,
 0.03386633, -0.8707283 , -0.0642144 , -0.5261853 , -0.5366441 ,
 -0.32214195, -0.27079347, 0.36429778, -0.04926702, -0.06325061,
 -0.4224059 , -0.04141013], dtype=float32),
 'gaiden': array([ 0.03694442, -0.00840622, 0.40425175, 0.1387582 , -0.23771454,
 0.16233376, 0.03707598, -0.5078011 , 0.23291597, 0.04278218,
 0.22757229, -0.04662805, 0.73560065, 0.0654904 , -0.3404447 ,
 -0.27189517, -0.02874799, -0.04407474, -0.13808799, -0.07573457,
 0.3978408 , -0.64466333, -0.06974618, -0.27431408, -0.08188342,
 0.07534437, -0.2436077 , 0.21145846, 0.125808 , 0.25228682,
 0.38125023, 0.43964297], dtype=float32),
 'protegido': array([ 0.56665003, 0.02630041, 0.79432154, -0.1659684 , -0.2770339 ,
 0.5420451 , -0.13389336, -0.5671256 , -0.03399916, -0.3362539 ,
 -0.20322956, 0.5491727 , -0.07410609, -0.6306755 , 0.2925735 ,
 -1.422273 , -0.14492823, -0.31507894, -0.26647982, 0.50701994,
 0.6172955 , -0.341631 , -0.11572693, 0.22890377, 0.55911905,
 -0.02285498, -0.24403843, -0.04549028, 0.20241591, 0.25729093,
 0.06621232, 0.2978325 ], dtype=float32),
 'advertencia

In [ ]:
songs_df.show(5)

+--------------------+--------------------+--------------------+
 track_uri| lyrics| words|
+--------------------+--------------------+--------------------+
spotify:track:003...|I wear it like a ...|[i, wear, it, lik...|
spotify:track:00J...|Meglio che ti lib...|[meglio, che, ti,...|
spotify:track:00J...|Only time until,\...|[only, time, unti...|
spotify:track:00W...|I'd like to thank...|[id, like, to, th...|
spotify:track:00X...|So why the ghetto...|[so, why, the, gh...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

In [ ]:
word_vecs['issues']

Out[30]: array([ 0.14932686, 0.37045252, 0.36223352, -0.45479566, 0.32517248,
 -0.30644256, -0.00156411, 0.63156086, 0.4372439 , 0.02330781,
 -0.01732084, -0.08565131, -0.05843329, 0.1574336 , -0.39021763,
 0.10594238, -0.54023653, -0.4181685 , -0.29182017, 0.27250916,
 0.72266924, -0.0846188 , -0.09610163, 0.06633095, -0.44312337,
 -0.09470002, -0.6399128 , 0.11257717, 0.14065027, -0.01737747,
 -0.0918657 , 0.44279173], dtype=float32)

In [ ]:
# Compute average embedding for each song
import numpy as np

def get_song_avg(words, word_vecs):
    all_vecs = [word_vecs[w] for w in words if w in word_vecs]
    if len(all_vecs) > 1:
        return np.mean(all_vecs, axis=0).tolist()

words2songvec = udf(lambda row: get_song_avg(row, word_vecs), returnType=ArrayType(FloatType()))

song_embs = songs_df.select("track_uri", "lyrics", "words", words2songvec(songs_df['words']).alias("embedding"))

In [ ]:
song_embs.show(5)

+--------------------+--------------------+--------------------+--------------------+
 track_uri| lyrics| words| embedding|
+--------------------+--------------------+--------------------+--------------------+
spotify:track:003...|I wear it like a ...|[i, wear, it, lik...|[-0.16906185, -0....|
spotify:track:00J...|Meglio che ti lib...|[meglio, che, ti,...|[0.17590189, 0.23...|
spotify:track:00J...|Only time until,\...|[only, time, unti...|[-0.16405874, -0....|
spotify:track:00W...|I'd like to thank...|[id, like, to, th...|[-0.09847277, -0....|
spotify:track:00X...|So why the ghetto...|[so, why, the, gh...|[-0.12120569, -0....|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [ ]:
song_embs.count()

Out[33]: 138123

In [ ]:
# Uncomment below to write lyrics vectors to new collection in MongoDB
# collection = 'lyrics_vectors'
# connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
# song_embs.write.format("mongo").option("uri", connection_string).mode("append").save()

## Computing lyrical distance between songs

In an effort to understand how similar two songs are *lyrically*, we will take the Euclidean distance between the lyrical embeddings for all pairs of songs in the dataset. We will later compare this distance to the song embeddings calculated by all songs' co-occurrence in playlists. In this section we take the following steps:

* Read lyrical embeddings DataFrame back from MongoDB
* Get new dataframe with all pairs of songs
* Merge this dataframe with the lyrical embeddings of each song
* Calculate the distance between the two lyrical embeddings for each row

In [ ]:
# Read lyrical embeddings from MongoDB
collection = 'lyrics_vectors'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
lyrics_df = spark.read.format('mongo').option("uri", connection_string).load()

In [ ]:
lyrics_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
 _id| embedding| lyrics| track_uri| words|
+--------------------+--------------------+--------------------+--------------------+--------------------+
{622459103db08d29...|[-0.0680133402347...|get me in so much...|spotify:track:019...|[get, me, in, so,...|
{622459103db08d29...|[-0.0028278413228...|I am neon\nCold n...|spotify:track:02S...|[i, am, neon, col...|
{622459103db08d29...|[-0.3783115446567...|Nin mun kèr’i la ...|spotify:track:038...|[nin, mun, kèri, ...|
{622459103db08d29...|[-0.0389421209692...|Since you've been...|spotify:track:054...|[since, youve, be...|
{622459103db08d29...|[0.09178527444601...|The pipes, the pi...|spotify:track:05M...|[the, pipes, the,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [ ]:
lyrics_df = lyrics_df.select("track_uri", "embedding")
lyrics_df.show(5)

+--------------------+--------------------+
 track_uri| embedding|
+--------------------+--------------------+
spotify:track:019...|[-0.0680133402347...|
spotify:track:02S...|[-0.0028278413228...|
spotify:track:038...|[-0.3783115446567...|
spotify:track:054...|[-0.0389421209692...|
spotify:track:05M...|[0.09178527444601...|
+--------------------+--------------------+
only showing top 5 rows

In [ ]:
lyrics_df.count()

Out[38]: 138123

In [ ]:
# Get all song, song pairs in dataframe
pairs = lyrics_df.alias("left")\
         .join(lyrics_df.alias("right"), col("left.track_uri") < col("right.track_uri"), "inner")\
         .select(col("left.track_uri").alias("track_1"), col("right.track_uri").alias("track_2"))
pairs.count()

Out[39]: 9538912269

Initially, I attempted to run the analysis on all song, song pairs in the dataframe above (9.5 billion observations). This proved too computationally intensive, slow, and ultimately expensive. Instead, we will proceed by sampling 0.1% of song pairs, which will still give us a sizeable dataset of about 9.5 million pairs.

In [ ]:
# Running the analysis on 10 billion word pairs is too slow, cumbersome, and expensive
# Instead, sub-sample pairs dataframe 0.1% of dataframe (about 10M pairs)
print(10_000_000 / 9_538_912_269)

pairs = pairs.sample(fraction=0.001, seed=42)
pairs.count()

0.001048337558622744
Out[40]: 9542654

In [ ]:
pairs.show(5)

+--------------------+--------------------+
 track_1| track_2|
+--------------------+--------------------+
spotify:track:1Hb...|spotify:track:1JF...|
spotify:track:019...|spotify:track:228...|
spotify:track:019...|spotify:track:4oM...|
spotify:track:019...|spotify:track:0PJ...|
spotify:track:019...|spotify:track:2zq...|
+--------------------+--------------------+
only showing top 5 rows

Next, we join our pairs dataframe with the lyrical embeddings dataframe to add the lyrical embeddings for each track.

In [ ]:
# Add track 1's lyrical embedding
pairs_embs = pairs.alias("left")\
                 .join(lyrics_df.alias("right"), col("left.track_1") == col("right.track_uri"), "left")\
                 .select(col("left.track_1").alias("track_1"), col("left.track_2").alias("track_2"), 
                         col("right.embedding").alias("track_1_lyrics_emb"))

In [ ]:
# Add track 2's lyrical embedding
pairs_embs = pairs_embs.alias("left")\
                 .join(lyrics_df.alias("right"), col("left.track_2") == col("right.track_uri"), "left")\
                 .select(col("left.track_1").alias("track_1"), col("left.track_2").alias("track_2"), 
                         col("left.track_1_lyrics_emb").alias("track_1_lyrics_emb"), col("right.embedding").alias("track_2_lyrics_emb"))
pairs_embs.show(10)

+--------------------+--------------------+--------------------+--------------------+
 track_1| track_2| track_1_lyrics_emb| track_2_lyrics_emb|
+--------------------+--------------------+--------------------+--------------------+
spotify:track:0Fr...|spotify:track:1ar...|[-0.0369906239211...|[-0.0518029890954...|
spotify:track:02L...|spotify:track:2Bn...|[-0.0070834341458...|[-0.0269077904522...|
spotify:track:0pi...|spotify:track:3eE...|[0.16496370732784...|[-0.0343168675899...|
spotify:track:3SV...|spotify:track:3mB...|[0.00585289811715...|[0.04230643436312...|
spotify:track:3Ow...|spotify:track:3uv...|[0.00444365013390...|[-0.0629589706659...|
spotify:track:1kY...|spotify:track:56E...|[-7.7373511157929...|[-0.0970294177532...|
spotify:track:1kY...|spotify:track:5jL...|[-7.7373511157929...|[0.14133681356906...|
spotify:track:02k...|spotify:track:6BK...|[-0.0214936435222...|[-0.0721251070499...|
spotify:track:2j7...|spotify:track:6Xm...|[0.14461068809032...|[-0.0324538648128...|
spotify:track:0mg...|spotify:track:7gB...|[-0.0287609733641...|[-0.0028592031449...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [ ]:
pairs_embs.printSchema()

root
-- track_1: string (nullable = true)
-- track_2: string (nullable = true)
-- track_1_lyrics_emb: array (nullable = true)
 |-- element: double (containsNull = true)
-- track_2_lyrics_emb: array (nullable = true)
 |-- element: double (containsNull = true)

Somehow, the pipeline above produces nulls for the lyrics embeddings as shown in the schema above (these could be created in the Word2Vec pipeline and read from MongoDB). We verify if there are in fact nulls below:

In [ ]:
pairs_embs.filter("track_1_lyrics_emb is NULL").show()

+--------------------+--------------------+------------------+--------------------+
 track_1| track_2|track_1_lyrics_emb| track_2_lyrics_emb|
+--------------------+--------------------+------------------+--------------------+
spotify:track:02t...|spotify:track:0CB...| null|[-0.0920922830700...|
spotify:track:00t...|spotify:track:0Wk...| null|[3.09337997350667...|
spotify:track:04E...|spotify:track:0bK...| null|[-0.0686664879322...|
spotify:track:0KO...|spotify:track:0y3...| null|[-0.0103816539049...|
spotify:track:0DH...|spotify:track:133...| null|[0.18182462453842...|
spotify:track:0DH...|spotify:track:170...| null|[-0.0514278747141...|
spotify:track:00H...|spotify:track:1d8...| null|[-0.0513060539960...|
spotify:track:04E...|spotify:track:26r...| null|[-0.0604121275246...|
spotify:track:0DH...|spotify:track:27k...| null|[0.02145343460142...|
spotify:track:08u...|spotify:track:2Jp...| null|[-0.0656279847025...|
spotify:track:0Sb...|spotify:track:2fc...| null|[-0.0305886175483...|
spotify:track:00t...|spotify:track:3Z5...| null|[0.19431520998477...|
spotify:track:08u...|spotify:track:4Pi...| null|[0.01913289353251...|
spotify:track:08u...|spotify:track:4XI...| null|[-0.0057331896387...|
spotify:track:04E...|spotify:track:5Hv...| null|[-0.0645982697606...|
spotify:track:0AE...|spotify:track:5L6...| null|[-0.0225361920893...|
spotify:track:0Sb...|spotify:track:5Qy...| null|[-0.0632697120308...|
spotify:track:04E...|spotify:track:5gu...| null|[-0.0839612185955...|
spotify:track:0AE...|spotify:track:6jp...| null|[-0.0447404533624...|
spotify:track:02t...|spotify:track:6oz...| null|[-0.0152486367151...|
+--------------------+--------------------+------------------+--------------------+
only showing top 20 rows

In [ ]:
# Drop rows with nulls in any column
pairs_embs = pairs_embs.na.drop("any")
pairs_embs.filter("track_1_lyrics_emb is NULL").show()

+-------+-------+------------------+------------------+
track_1|track_2|track_1_lyrics_emb|track_2_lyrics_emb|
+-------+-------+------------------+------------------+
+-------+-------+------------------+------------------+

In [ ]:
pairs_embs.filter("track_2_lyrics_emb is NULL").show()

+-------+-------+------------------+------------------+
track_1|track_2|track_1_lyrics_emb|track_2_lyrics_emb|
+-------+-------+------------------+------------------+
+-------+-------+------------------+------------------+

In [ ]:
# Check processed data size
pairs_embs.count()

Out[48]: 9458354

In [ ]:
pairs_embs.show(10)

+--------------------+--------------------+--------------------+--------------------+
 track_1| track_2| track_1_lyrics_emb| track_2_lyrics_emb|
+--------------------+--------------------+--------------------+--------------------+
spotify:track:00L...|spotify:track:00e...|[-0.0721510648727...|[-0.0244157034903...|
spotify:track:00I...|spotify:track:00e...|[-0.0179049242287...|[-0.0244157034903...|
spotify:track:006...|spotify:track:01V...|[-0.0687494128942...|[0.05495329573750...|
spotify:track:017...|spotify:track:01b...|[-0.0548606030642...|[-0.1526609361171...|
spotify:track:015...|spotify:track:01b...|[0.03307503461837...|[-0.1526609361171...|
spotify:track:015...|spotify:track:030...|[-0.0162511579692...|[-0.1133899688720...|
spotify:track:00K...|spotify:track:03G...|[-0.0602876096963...|[-0.0122050959616...|
spotify:track:02Q...|spotify:track:03J...|[0.10781918466091...|[0.05152385681867...|
spotify:track:01P...|spotify:track:03Y...|[-0.0749082714319...|[0.00584124401211...|
spotify:track:006...|spotify:track:03Y...|[-0.0415595024824...|[0.00584124401211...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

Victory! We were able to remove the null values and still maintain 9.46 million rows. Below, we calculate the distance between each pair's lyrics embeddings.

In [ ]:
import numpy as np

def calculate_distance(arr_1, arr_2):
    # Calculate the distance between two lists.
    try:
        arr_1, arr_2 = np.array(arr_1), np.array(arr_2)
        return float(np.linalg.norm(arr_1 - arr_2))
    except:
        return -99999.99

# Define distance UDF and calculate lyrics distance --> new dataframe
distance = udf(lambda x, y: calculate_distance(x, y), FloatType())
lyrics_dist = pairs_embs.select("track_1", "track_2", 
                                distance(pairs_embs["track_1_lyrics_emb"],
                                         pairs_embs["track_2_lyrics_emb"]).alias("lyrics_distance")).cache()
lyrics_dist.show(10)

+--------------------+--------------------+---------------+
 track_1| track_2|lyrics_distance|
+--------------------+--------------------+---------------+
spotify:track:00Y...|spotify:track:00f...| 0.658865|
spotify:track:01E...|spotify:track:01U...| 0.30476514|
spotify:track:00l...|spotify:track:01V...| 0.4378901|
spotify:track:016...|spotify:track:01V...| 0.4129816|
spotify:track:01l...|spotify:track:02Q...| 0.4081815|
spotify:track:020...|spotify:track:02Q...| 0.44501796|
spotify:track:00W...|spotify:track:02R...| 0.34712142|
spotify:track:02Q...|spotify:track:030...| 0.5778875|
spotify:track:01p...|spotify:track:030...| 0.5344382|
spotify:track:006...|spotify:track:03G...| 0.3431269|
+--------------------+--------------------+---------------+
only showing top 10 rows

In [ ]:
lyrics_dist.count()

Out[51]: 9461314

## Load song embeddings from MongoDB

Previously, Chandrish computed song embeddings using Word2Vec, which he saved in MongoDB. We will load this into a Spark DataFrame and join with the lyrics distance DataFrame.

In [ ]:
# Read song vectors from MongoDB
collection = 'song_vectors'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
song_embs = spark.read.format('mongo').option("uri", connection_string).load().select("track_uri", "values")
song_embs.show(5)

+--------------------+--------------------+
 track_uri| values|
+--------------------+--------------------+
spotify:track:5EC...|[0.35091972351074...|
spotify:track:6Tr...|[-0.1229294389486...|
spotify:track:5yf...|[0.02834359928965...|
spotify:track:0Ld...|[0.12241361290216...|
spotify:track:1Yl...|[-0.1754425019025...|
+--------------------+--------------------+
only showing top 5 rows

In [ ]:
# Join with lyrical embeddings df (track 1)
combined = lyrics_dist.alias("left")\
                .join(song_embs.alias("right"), col("left.track_1") == col("right.track_uri"), "left")\
                .select(col("left.track_1").alias("track_1"), col("left.track_2").alias("track_2"),
                        col("left.lyrics_distance").alias("lyrics_distance"), col("right.values").alias("track_1_song_emb"))
combined.show(5)

+--------------------+--------------------+---------------+--------------------+
 track_1| track_2|lyrics_distance| track_1_song_emb|
+--------------------+--------------------+---------------+--------------------+
spotify:track:002...|spotify:track:01j...| 0.36169654|[-0.3095772266387...|
spotify:track:006...|spotify:track:00r...| 2.9032712|[-0.0910839736461...|
spotify:track:006...|spotify:track:00N...| 2.5462215|[-0.0910839736461...|
spotify:track:008...|spotify:track:00O...| 0.35543087|[0.01410138141363...|
spotify:track:008...|spotify:track:00M...| 0.3265417|[0.27851784229278...|
+--------------------+--------------------+---------------+--------------------+
only showing top 5 rows

In [ ]:
# Join with track 2 embedding
combined = combined.alias("left")\
                .join(song_embs.alias("right"), col("left.track_2") == col("right.track_uri"), "left")\
                .select(col("left.track_1").alias("track_1"), 
                        col("left.track_2").alias("track_2"),
                        col("left.lyrics_distance").alias("lyrics_distance"),
                        col("left.track_1_song_emb").alias("track_1_song_emb"),
                        col("right.values").alias("track_2_song_emb"))
combined.show(5)

+--------------------+--------------------+---------------+--------------------+--------------------+
 track_1| track_2|lyrics_distance| track_1_song_emb| track_2_song_emb|
+--------------------+--------------------+---------------+--------------------+--------------------+
spotify:track:00Y...|spotify:track:00f...| 0.658865|[-0.0548955425620...|[-0.1701245158910...|
spotify:track:01E...|spotify:track:01U...| 0.30476514|[0.28122106194496...|[-0.4792552292346...|
spotify:track:016...|spotify:track:01V...| 0.4129816|[0.29193156957626...|[0.00648248475044...|
spotify:track:00l...|spotify:track:01V...| 0.4378901|[-0.0685095563530...|[0.00648248475044...|
spotify:track:020...|spotify:track:02Q...| 0.44501796|[-0.7943779230117...|[-0.8351961970329...|
+--------------------+--------------------+---------------+--------------------+--------------------+
only showing top 5 rows

In [ ]:
# Compute song vector distances
ml_df = combined.select("track_1", "track_2", "lyrics_distance",
                          distance(combined["track_1_song_emb"],
                                   combined["track_2_song_emb"]).alias("song_distance")).cache()
ml_df.show()

+--------------------+--------------------+---------------+-------------+
 track_1| track_2|lyrics_distance|song_distance|
+--------------------+--------------------+---------------+-------------+
spotify:track:00Y...|spotify:track:00f...| 0.658865| 2.9069214|
spotify:track:01E...|spotify:track:01U...| 0.30476514| 3.9558227|
spotify:track:016...|spotify:track:01V...| 0.4129816| 2.5016074|
spotify:track:00l...|spotify:track:01V...| 0.4378901| 2.8883207|
spotify:track:01l...|spotify:track:02Q...| 0.4081815| 3.2217238|
spotify:track:020...|spotify:track:02Q...| 0.44501796| 2.1954544|
spotify:track:00W...|spotify:track:02R...| 0.34712142| 2.99181|
spotify:track:02Q...|spotify:track:030...| 0.5778875| 3.4185224|
spotify:track:01p...|spotify:track:030...| 0.5344382| 2.3745182|
spotify:track:006...|spotify:track:03G...| 0.3431269| 2.8963737|
spotify:track:00p...|spotify:track:03X...| 0.2799113| 3.4106166|
spotify:track:00v...|spotify:track:03X...| 0.3607634| 3.5515358|
spotify:track:00j...|spotify:track:03X...| 0.2911445| 3.7463322|
spotify:track:03C...|spotify:track:03Y...| 0.48592055| 3.5625875|
spotify:track:00t...|spotify:track:03Y...| 0.33505994| 3.6853225|
spotify:track:01Z...|spotify:track:03b...| 3.6529765| 3.2617724|
spotify:track:00a...|spotify:track:040...| 0.3456389| 2.9332812|
spotify:track:01E...|spotify:track:040...| 0.4409738| 2.4641602|
spotify:track:01C...|spotify:track:04L...| 0.5208658| 4.8674474|
spotify:track:01s...|spotify:track:04L...| 0.24642082| 2.9985151|
+--------------------+--------------------+---------------+-------------+
only showing top 20 rows

In [ ]:
ml_df.count()

Out[56]: 9461314

In [ ]:
# Drop any NAs that may exist
ml_df = ml_df.na.drop("any").cache()
ml_df.count()

Out[57]: 9461314

In [ ]:
ml_df.show()

+--------------------+--------------------+---------------+-------------+
 track_1| track_2|lyrics_distance|song_distance|
+--------------------+--------------------+---------------+-------------+
spotify:track:00Y...|spotify:track:00f...| 0.658865| 2.9069214|
spotify:track:01E...|spotify:track:01U...| 0.30476514| 3.9558227|
spotify:track:016...|spotify:track:01V...| 0.4129816| 2.5016074|
spotify:track:00l...|spotify:track:01V...| 0.4378901| 2.8883207|
spotify:track:01l...|spotify:track:02Q...| 0.4081815| 3.2217238|
spotify:track:020...|spotify:track:02Q...| 0.44501796| 2.1954544|
spotify:track:00W...|spotify:track:02R...| 0.34712142| 2.99181|
spotify:track:02Q...|spotify:track:030...| 0.5778875| 3.4185224|
spotify:track:01p...|spotify:track:030...| 0.5344382| 2.3745182|
spotify:track:006...|spotify:track:03G...| 0.3431269| 2.8963737|
spotify:track:00p...|spotify:track:03X...| 0.2799113| 3.4106166|
spotify:track:00v...|spotify:track:03X...| 0.3607634| 3.5515358|
spotify:track:00j...|spotify:track:03X...| 0.2911445| 3.7463322|
spotify:track:03C...|spotify:track:03Y...| 0.48592055| 3.5625875|
spotify:track:00t...|spotify:track:03Y...| 0.33505994| 3.6853225|
spotify:track:01Z...|spotify:track:03b...| 3.6529765| 3.2617724|
spotify:track:00a...|spotify:track:040...| 0.3456389| 2.9332812|
spotify:track:01E...|spotify:track:040...| 0.4409738| 2.4641602|
spotify:track:01C...|spotify:track:04L...| 0.5208658| 4.8674474|
spotify:track:01s...|spotify:track:04L...| 0.24642082| 2.9985151|
+--------------------+--------------------+---------------+-------------+
only showing top 20 rows

## Linear regression

Finally, we're able to run the linear regression to see how much variance in song distance can be explained by changes in lyrics distance (or how much of song similarity is explained by lyrics similarity).

In [ ]:
# Create training and test data
splits = ml_df.randomSplit([0.8, 0.2], 1)
train = splits[0].cache()
valid = splits[1].cache()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

va = VectorAssembler(outputCol="features", inputCols=["lyrics_distance"])
train_va = va.transform(train).select("features", "song_distance").withColumnRenamed("song_distance", "label").cache()
valid_va = va.transform(valid).select("features", "song_distance").withColumnRenamed("song_distance", "label").cache()

In [ ]:
train_va.show()

+--------------------+---------+
 features| label|
+--------------------+---------+
[0.43925023078918...|3.6920695|
[0.3891196846961975]|3.9978712|
[0.4236740469932556]|2.9444509|
[0.4685608148574829]|3.5022442|
[3.1007516384124756]|4.9172974|
[2.2533645629882812]| 4.860533|
[3.0769996643066406]|4.1763406|
[3.1457955837249756]| 4.500409|
[3.1196353435516357]| 4.063187|
[3.1241977214813232]|3.8657582|
 [3.111677646636963]|3.7619135|
[3.1014678478240967]| 5.051492|
[0.5590356588363647]|2.7080197|
[3.1449007987976074]|2.9338021|
 [3.162168025970459]|3.6978698|
[3.1895618438720703]|2.5055172|
[0.4005802273750305]|3.6941667|
[2.4325225353240967]|4.3587627|
[0.42740556597709...|3.2281666|
[0.3492421805858612]|2.4606647|
+--------------------+---------+
only showing top 20 rows

In [ ]:
# Fit linear regression to data
lr = LinearRegression(regParam=0.0)
model = lr.fit(train_va)

In [ ]:
# Get validation predictions
valid_predictions = model.transform(valid_va)

In [ ]:
valid_predictions.show()

+--------------------+---------+------------------+
 features| label| prediction|
+--------------------+---------+------------------+
[3.1528313159942627]|3.6130002| 3.623285249556707|
 [3.098644971847534]|4.5739603| 3.615417318370832|
[0.41371408104896...| 2.719344|3.2255616855488443|
[0.2539471983909607]|2.9667482| 3.202363315953526|
[0.4732712209224701]|2.6499662| 3.234209463610479|
 [2.447709560394287]| 5.058909| 3.520900607483399|
[0.5843866467475891]|4.2461677| 3.250343575218265|
[0.2652985453605652]|2.8399422| 3.204011547038871|
[0.40662094950675...| 2.508804|3.2245317531600617|
[2.5963921546936035]| 5.353708|3.5424895232684657|
[0.4381134808063507]|3.6804113|3.2291045117219466|
[0.7053680419921875]| 3.203352| 3.267910239062168|
[2.5521392822265625]|4.3990326|3.5360639457390945|
[0.27468061447143...| 4.147687| 3.205373836289195|
[3.0692291259765625]|3.4542017|3.6111460973698284|
[0.32819145917892...|2.3393183|3.2131436840337946|
[0.6434056758880615]|3.1557837|3.2589132188664314|
[2.1604671478271484]|3.9004805|3.4791926168202316|
[2.4186792373657227]|3.6295848|3.5166853649257175|
[0.3432964086532593]|3.8927882| 3.215336943330055|
+--------------------+---------+------------------+
only showing top 20 rows

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_predictions)))

rmse: 0.7535408027933982

In [ ]:
evaluator.setMetricName("r2")
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_predictions)))

r2: 0.03442525194969537

In [ ]:
# Scale and run regression again
from pyspark.ml.feature import MinMaxScaler

mmScaler = MinMaxScaler(outputCol="lyrics_dist_scaled", inputCol="features")

lyrics_scaler = mmScaler.fit(train_va)

In [ ]:
print("Original lyrics (min, max):", (lyrics_scaler.originalMin, lyrics_scaler.originalMax))

Original lyrics (min, max): (DenseVector([0.0]), DenseVector([8.325]))

In [ ]:
row1 = ml_df.agg({"song_distance": "max"}).collect()[0]
print(row1)

Row(max(song_distance)=7.888247489929199)

In [ ]:
row1 = ml_df.agg({"song_distance": "min"}).collect()[0]
print(row1)

Row(min(song_distance)=0.3243006467819214)

In [ ]:
train_va_scaled = lyrics_scaler.transform(train_va)
train_va_scaled.show()

+--------------------+---------+--------------------+
 features| label| lyrics_dist_scaled|
+--------------------+---------+--------------------+
[0.43925023078918...|3.6920695|[0.05276277960550...|
[0.3891196846961975]|3.9978712|[0.04674109362880...|
[0.4236740469932556]|2.9444509|[0.05089176692273...|
[0.4685608148574829]|3.5022442|[0.05628356975859...|
[3.1007516384124756]|4.9172974|[0.3724625824670364]|
[2.2533645629882812]| 4.860533| [0.27067436616756]|
[3.0769996643066406]|4.1763406|[0.3696094930726601]|
[3.1457955837249756]| 4.500409| [0.377873265472976]|
[3.1196353435516357]| 4.063187|[0.3747308949289396]|
[3.1241977214813232]|3.8657582|[0.3752789282008826]|
 [3.111677646636963]|3.7619135|[0.37377501561677...|
[3.1014678478240967]| 5.051492|[0.37254861360985...|
[0.5590356588363647]|2.7080197|[0.06715141664424...|
[3.1449007987976074]|2.9338021|[0.37776578382218...|
 [3.162168025970459]|3.6978698|[0.3798399247966383]|
[3.1895618438720703]|2.5055172|[0.38313047281501...|
[0.4005802273750305]|3.6941667|[0.04811773510816...|
[2.4325225353240967]|4.3587627|[0.29219483888749...|
[0.42740556597709...|3.2281666|[0.05133999733887...|
[0.3492421805858612]|2.4606647|[0.0419510040326955]|
+--------------------+---------+--------------------+
only showing top 20 rows

In [ ]:
train_va_scaled = train_va_scaled.select("lyrics_dist_scaled", "label")\
                                 .withColumnRenamed("lyrics_dist_scaled", "features")\
                                 .cache()

In [ ]:
train_va_scaled.show()

+--------------------+---------+
 features| label|
+--------------------+---------+
[0.05276277960550...|3.6920695|
[0.04674109362880...|3.9978712|
[0.05089176692273...|2.9444509|
[0.05628356975859...|3.5022442|
[0.3724625824670364]|4.9172974|
 [0.27067436616756]| 4.860533|
[0.3696094930726601]|4.1763406|
 [0.377873265472976]| 4.500409|
[0.3747308949289396]| 4.063187|
[0.3752789282008826]|3.8657582|
[0.37377501561677...|3.7619135|
[0.37254861360985...| 5.051492|
[0.06715141664424...|2.7080197|
[0.37776578382218...|2.9338021|
[0.3798399247966383]|3.6978698|
[0.38313047281501...|2.5055172|
[0.04811773510816...|3.6941667|
[0.29219483888749...|4.3587627|
[0.05133999733887...|3.2281666|
[0.0419510040326955]|2.4606647|
+--------------------+---------+
only showing top 20 rows

In [ ]:
# Fit linear regression to *scaled* data
lr = LinearRegression(regParam=0.0)
model = lr.fit(train_va_scaled)

In [ ]:
valid_va_scaled = lyrics_scaler.transform(valid_va)\
                               .select("lyrics_dist_scaled", "label")\
                               .withColumnRenamed("lyrics_dist_scaled", "features")\
                               .cache()

In [ ]:
valid_preds_scaled = model.transform(valid_va_scaled)

In [ ]:
# Get results
evaluator = RegressionEvaluator()
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_preds_scaled)))
evaluator.setMetricName("r2")
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_preds_scaled)))

rmse: 0.753540802793398
r2: 0.034425251949696034

So far linear regression seems to suggest no relationship on both scaled and unscaled lyrical distances. Next, fit with a Huber loss function and see if any better results arise.

In [ ]:
# Fit with huber loss
lr_huber = LinearRegression(regParam=0.0, loss="huber")
model_huber = lr_huber.fit(train_va_scaled)

In [ ]:
# Predict on validation
huber_preds = model_huber.transform(valid_va_scaled)
# Evaluate and print results
evaluator = RegressionEvaluator()
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_preds_scaled)))
evaluator.setMetricName("r2")
print(evaluator.getMetricName() + ": " + str(evaluator.evaluate(valid_preds_scaled)))

rmse: 0.753540802793398
r2: 0.034425251949696034

## Conclusion

Clearly the lyrics distances can only account for about 3% of the variance in the distance between songs. This intuitively makes sense when you consider the range of themes, dialects, and languages that often coincide in the same playlist.

Future considerations of this question may weight the word embeddings by TFIDF score in the averaging process for lyrical embeddings. Also, other tree-based models or radius-based models (KNN regressors) could be implemented in case the relationship between lyrics and song placement is not linear.

In truth, the lyrics embedding should be fed as input to a much more sophisticated model that takes into account other metrics in order to produce anything meaningful.

In [ ]:
spark.stop()